In [31]:
import numpy as np
import pandas as pd
import json
from geopy.distance import great_circle
import folium
from folium import plugins

**Load Border Points Data:**  
*Read the JSON file containing the coordinates of border points.*

In [2]:
with open("Border_Points.json", "r") as file:
    data = json.load(file)

In [3]:
data = data["features"][0]["geometry"]["coordinates"][0]
len(data)

2284

**Coordinate Regions Distribution:**  
   *Calculate the distribution of points within each of the four coordinate quadrants.*


In [4]:
Sari_coor   = [36.561151, 53.060088] # Coordinates that demarcate the division between Iran's eastern and western regions
Ilam_coor = [33.568375, 46.045389]  # Coordinates that demarcate the division between Iran's northern and southern regions

In [5]:
Ilam_lat = Ilam_coor[0]
Ilam_lng = Ilam_coor[1]
Sari_lat = Sari_coor[0]
Sari_lng = Sari_coor[1]

In [6]:
points = pd.DataFrame(data, columns=["lng", "lat"])
points.head()

,lng,lat
0,48.613949,29.928497
1,48.621193,29.934804
2,48.624422,29.934589
3,48.630881,29.932651
4,48.640818,29.932005


In [7]:
len_NE = len(points.loc[(points["lat"] > Ilam_lat) & (points["lng"] > Sari_lng)])
len_NW = len(points.loc[(points["lat"] > Ilam_lat) & (points["lng"] < Sari_lng)])
len_SW = len(points.loc[(points["lat"] < Ilam_lat) & (points["lng"] < Sari_lng)])
len_SE = len(points.loc[(points["lat"] < Ilam_lat) & (points["lng"] > Sari_lng)])
print(f"len_NE : {len_NE}")
print(f"len_NW : {len_NW}")
print(f"len_SW : {len_SW}")
print(f"len_SE : {len_SE}")

len_NE : 94
len_NW : 446
len_SW : 768
len_SE : 976


**Northeast Region Completion:**  
   *Populate the dataset with points located in the Northeast (NE) region.*


In [8]:
with open("NE_Complete.json", "r") as file:
    NE_Completoin_data = json.load(file)

In [9]:
NE_Completion = list()

for feature in NE_Completoin_data["features"]:
    NE_Completion.append(feature["geometry"]["coordinates"][:])

NE_Completion = [element for sublist in NE_Completion for element in sublist]

In [10]:
NE_Completion = pd.DataFrame(NE_Completion, columns=["lng", "lat"])

In [11]:
len(NE_Completion)

923

In [12]:
NE_Completion.head()

,lng,lat
0,60.551479,33.640504
1,60.551331,33.643335
2,60.551035,33.645919
3,60.551183,33.648627
4,60.551183,33.651580


**Northwest Region Completion:**  
   *Append points to the dataset within the Northwest (NW) region.*

In [13]:
with open("NW_Complete.json", "r") as file:
    NW_Completoin_data = json.load(file)

In [14]:
NW_Completion = list()

for feature in NW_Completoin_data["features"]:
    NW_Completion.append(feature["geometry"]["coordinates"][:])

NW_Completion = [element for sublist in NW_Completion for element in sublist]

In [15]:
NW_Completion = pd.DataFrame(NW_Completion, columns=["lng", "lat"])

In [16]:
len(NW_Completion)

552

In [17]:
NW_Completion.head()

,lng,lat
0,45.924524,33.598682
1,45.912369,33.620953
2,45.902645,33.641194
3,45.844300,33.627025
4,45.810265,33.610830


**Southwest Region Completion:**  
   *Add points to the dataset situated in the Southwest (SW) region.*

In [18]:
with open("SW_Complete.json", "r") as file:
    SW_Completoin_data = json.load(file)

In [19]:
SW_Completion = list()

for feature in SW_Completoin_data["features"]:
    SW_Completion.append(feature["geometry"]["coordinates"][:])

SW_Completion = [element for sublist in SW_Completion for element in sublist]

In [20]:
SW_Completion = pd.DataFrame(SW_Completion, columns=["lng", "lat"])

In [21]:
len(SW_Completion)

216

In [22]:
SW_Completion.head()

,lng,lat
0,54.007617,26.755803
1,53.954357,26.735419
2,53.885879,26.708235
3,53.802184,26.721828
4,53.741315,26.701438


**Dataframes Consolidation:**  
   *Combine different dataframes into one unified structure.*

In [23]:
points = pd.concat([points, NE_Completion, NW_Completion, SW_Completion], ignore_index=True)
points

,lng,lat
0,48.613949,29.928497
1,48.621193,29.934804
2,48.624422,29.934589
3,48.630881,29.932651
4,48.640818,29.932005
...,...,...
3970,46.169003,33.069856
3971,45.866036,33.496451
3972,45.946185,33.548421
3973,45.913532,33.630024


**Reevaluating Distribution:**  
   *Assess the distribution of points across the four coordinate quadrants again for accuracy.*

In [24]:
len_NE = len(points.loc[(points["lat"] > Ilam_lat) & (points["lng"] > Sari_lng)])
len_NW = len(points.loc[(points["lat"] > Ilam_lat) & (points["lng"] < Sari_lng)])
len_SW = len(points.loc[(points["lat"] < Ilam_lat) & (points["lng"] < Sari_lng)])
len_SE = len(points.loc[(points["lat"] < Ilam_lat) & (points["lng"] > Sari_lng)])
print(f"len_NE : {len_NE}")
print(f"len_NW : {len_NW}")
print(f"len_SW : {len_SW}")
print(f"len_SE : {len_SE}")

len_NE : 1017
len_NW : 1000
len_SW : 963
len_SE : 995


**Center Optimization:**  
   *Optimize and locate the precise geographic center of Iran.*

In [25]:
# Define geographic bounding box edges for the search area
lng_min = 49.781853529617024
lng_max = 58.37602322566812
lat_min = 29.235933551806966
lat_max = 35.393730146031245

In [26]:
# Create a copy of an existing points DataFrame for manipulation
points_copy = points.copy()

In [27]:
# Initialize a list to store potential center coordinates
center_candidates = []

# Initialize latitude and longitude candidates for the center point
lat_candidate = 0
lng_candidate = 0

In [28]:
# Define initial step size for the grid search
step = 5

# Set the initial value for minimum variance high enough to be iteratively reduced
# (Actually, this represents the variance of the distances between all border points
# and the one at the coordinates with the current lat_min and lng_min values.)
min_variance = 145231.6822812455

# Start a while loop to run until the step size becomes sufficiently small for precision
while step > 0.00000000000001:
    print(step)  # Print current step size for tracking

    # Nested loop to perform a grid search within the bounding box
    for lng in np.arange(lng_min, lng_max, step):
        for lat in np.arange(lat_min, lat_max, step):

            # Calculate distance of all points from current center candidate using great circle formula
            points["distances"] = points.apply(lambda row: great_circle((row["lat"], row["lng"]), (lat, lng)).kilometers, axis=1)
            
            # Compute the variance of the distances to evaluate the dispersion
            variance = points["distances"].var(ddof=0)

            # Update minimum variance and candidates if current variance is lower than the previous
            if variance < min_variance:
                min_variance = variance
                lat_candidate = lat
                lng_candidate = lng

    # Add the best center coordinates for this iteration to the candidate list
    center_candidates.append([lat_candidate, lng_candidate])

    # Update the bounding box edges centered around the new candidate
    lat_min = lat_candidate - step
    lat_max = lat_candidate + step
    lng_min = lng_candidate - step
    lng_max = lng_candidate + step

    # Reduce the step size by a factor of 10 for the next iteration
    step /= 10

    # Print the latest center candidate and step size for verification
    print(lat_candidate)
    print(lng_candidate)
    print(7*"=")


5
34.235933551806966
54.781853529617024
0.5
31.735933551806966
54.781853529617024
0.05
31.835933551806974
54.731853529617
0.005
31.830933551806964
54.73685352961703
0.0005
31.83193355180695
54.73835352961706
5e-05
31.831883551806968
54.738453529617075
5e-06
31.831893551806985
54.73842852961708
5.000000000000001e-07
31.831892051806975
54.738429529617065
5.000000000000001e-08
31.83189205180698
54.73842942961704
5.000000000000001e-09
31.831892051806985
54.73842941961702
5.000000000000001e-10
31.831892051806985
54.73842941961702
5.0000000000000015e-11
31.831892051806985
54.73842941961702
5.000000000000001e-12
31.831892051806985
54.73842941961702
5.000000000000001e-13
31.831892051806985
54.73842941961702
5.000000000000001e-14
31.831892051806985
54.73842941961702


In [29]:
len(center_candidates)

15

**Mapping Points Visualization:**  
   *Display the points on a map for visual reference.*

In [34]:
mymap = folium.Map(location=center_candidates[0], zoom_start=6)

for tag, coords in enumerate(center_candidates[:10]):
    icon = folium.plugins.BeautifyIcon(
        # icon='circle',
        icon=tag,
        border_color='black',
        text_color='black',
        inner_icon_style='margin-top:5;',
        icon_shape='marker'
    )
    folium.Marker(
        location=[coords[0], coords[1]],
        popup=folium.Popup(str(tag), parse_html=True),
        icon=icon,
        tooltip=str(tag)
    ).add_to(mymap)


mymap